This notebook implements language model to model text

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

In [8]:
import data
from importlib import reload
#reload(data)

In [15]:
reload(data)

<module 'data' from '/home/manoj/Documents/github/language-modelling/data.py'>

## Load data

In [9]:
import os

In [12]:
work_dir = os.getcwd()
data_path = os.path.join(work_dir, 'data/wikitext-2-raw/')

In [16]:
corpus = data.Corpus(data_path)

In [47]:
batch_size = 20
eval_batch_size = 10

In [43]:
def batchify(data, batch_size):
    num_batches = data.size(0) // batch_size
    data = data.narrow(0, 0, num_batches * batch_size)
    data = data.view(batch_size, -1).t().contiguous()
    return data.cuda()
    

In [44]:
train_data = batchify(corpus.train, batch_size)

In [45]:
type(train_data)

torch.cuda.LongTensor

In [48]:
valid_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

## Modelling

In [59]:
class RNNModel(nn.Module):
    def __init__(self, num_tokens, emb_inp, num_hidden, num_layers):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(0.2)
        self.encoder = nn.Embedding(num_tokens, emb_inp)
        self.rnn = nn.RNN(emb_inp, num_hidden, nonlinearity='tanh', dropout = 0.2)
        self.decoder = nn.Linear(num_hidden, num_tokens)
        
        self.init_weights()
        self.num_hidden = num_hidden
        self.num_layers = num_layers
    
    def init_weights(self):
        self.encoder.weight.data.uniform_(-0.1, 0.1)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-0.1, 0.1)
        
    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

In [60]:
num_tokens = len(corpus.dictionary.idx2word)
emb_size = 200
num_hidden = 200
num_layers = 2

In [61]:
model = RNNModel(num_tokens, emb_size, num_hidden, num_layers)

In [53]:
len(corpus.dictionary.idx2word)

84608